# Flood Risk Exploratory Data Analysis

This notebook explores NFIP claims, policies, and disaster declaration data
from the OpenFEMA API to understand patterns in flood risk and insurance coverage.

In [ ]:
import sys
sys.path.insert(0, '..')

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from src.ingestion.openfema_client import OpenFEMAClient

## 1. Load Data

Fetch a sample of claims, policies, and disaster declarations from OpenFEMA.
Adjust `max_records` to control the sample size.

In [ ]:
client = OpenFEMAClient()

# Fetch sample data (increase max_records for deeper analysis)
claims = client.fetch_claims(max_records=10000)
policies = client.fetch_policies(max_records=10000)
disasters = client.fetch_disasters(max_records=5000)

print(f"Claims:    {claims.shape}")
print(f"Policies:  {policies.shape}")
print(f"Disasters: {disasters.shape}")

## 2. Claims Analysis

In [ ]:
claims.head()

In [ ]:
claims.info()

In [ ]:
# Total claim payouts by year
if 'yearOfLoss' in claims.columns and 'amountPaidOnBuildingClaim' in claims.columns:
    claims['totalPaid'] = (
        claims['amountPaidOnBuildingClaim'].fillna(0) +
        claims['amountPaidOnContentsClaim'].fillna(0)
    )
    yearly = claims.groupby('yearOfLoss').agg(
        total_paid=('totalPaid', 'sum'),
        claim_count=('totalPaid', 'count'),
        avg_claim=('totalPaid', 'mean'),
    ).reset_index()

    fig = px.bar(
        yearly, x='yearOfLoss', y='total_paid',
        title='Total NFIP Claim Payouts by Year',
        labels={'yearOfLoss': 'Year', 'total_paid': 'Total Paid ($)'},
    )
    fig.show()

In [ ]:
# Claims by state
if 'state' in claims.columns:
    state_claims = claims.groupby('state').agg(
        claim_count=('state', 'count'),
        total_paid=('totalPaid', 'sum'),
    ).reset_index().sort_values('claim_count', ascending=False)

    fig = px.bar(
        state_claims.head(20), x='state', y='claim_count',
        title='Top 20 States by Claim Count',
        labels={'state': 'State', 'claim_count': 'Number of Claims'},
    )
    fig.show()

In [ ]:
# Claims by flood zone
if 'floodZone' in claims.columns:
    zone_claims = claims.groupby('floodZone').size().reset_index(name='count')
    zone_claims = zone_claims.sort_values('count', ascending=False).head(15)

    fig = px.bar(
        zone_claims, x='floodZone', y='count',
        title='Claims by Flood Zone',
        labels={'floodZone': 'Flood Zone', 'count': 'Number of Claims'},
    )
    fig.show()

## 3. Policy Analysis

In [ ]:
policies.head()

In [ ]:
# Policy counts by state
state_col = 'propertyState' if 'propertyState' in policies.columns else 'state'
if state_col in policies.columns:
    policy_col = 'policyCount' if 'policyCount' in policies.columns else state_col
    if 'policyCount' in policies.columns:
        state_policies = policies.groupby(state_col)['policyCount'].sum().reset_index()
    else:
        state_policies = policies.groupby(state_col).size().reset_index(name='policyCount')
    state_policies = state_policies.sort_values('policyCount', ascending=False)

    fig = px.bar(
        state_policies.head(20), x=state_col, y='policyCount',
        title='Top 20 States by Policy Count',
        labels={state_col: 'State', 'policyCount': 'Policies'},
    )
    fig.show()

## 4. Disaster Declarations

In [ ]:
disasters.head()

In [ ]:
# Flood-related disaster declarations over time
if 'incidentType' in disasters.columns:
    flood_disasters = disasters[disasters['incidentType'].str.contains('Flood', case=False, na=False)]
    if 'declarationDate' in flood_disasters.columns:
        flood_disasters = flood_disasters.copy()
        flood_disasters['year'] = pd.to_datetime(flood_disasters['declarationDate']).dt.year
        yearly_disasters = flood_disasters.groupby('year').size().reset_index(name='count')

        fig = px.line(
            yearly_disasters, x='year', y='count',
            title='Flood-Related Federal Disaster Declarations per Year',
            labels={'year': 'Year', 'count': 'Declarations'},
        )
        fig.show()

## 5. Coverage Gap Preview

Quick look at the ratio of claims to policies by state — a simple proxy for coverage gaps.

In [ ]:
# Merge claims and policy counts by state for a quick coverage gap view
if 'state' in claims.columns and state_col in policies.columns:
    claims_by_state = claims.groupby('state').size().reset_index(name='claim_count')
    if 'policyCount' in policies.columns:
        policies_by_state = policies.groupby(state_col)['policyCount'].sum().reset_index()
        policies_by_state.columns = ['state', 'policy_count']
    else:
        policies_by_state = policies.groupby(state_col).size().reset_index(name='policy_count')
        policies_by_state.columns = ['state', 'policy_count']

    gap = claims_by_state.merge(policies_by_state, on='state', how='outer').fillna(0)
    gap['claims_per_policy'] = gap['claim_count'] / gap['policy_count'].replace(0, 1)
    gap = gap.sort_values('claims_per_policy', ascending=False)

    fig = px.bar(
        gap.head(20), x='state', y='claims_per_policy',
        title='Claims-to-Policy Ratio by State (Higher = Bigger Gap)',
        labels={'state': 'State', 'claims_per_policy': 'Claims per Policy'},
    )
    fig.show()

    gap.head(20)